# Collecting DES Data

This notebook arranges DES data into 204 csv files across the entire 5,000 square degrees of coverage. The tiles are crossmatched to VHS and CatWise2020. These matches form the collection of sources from which candidates are pulled.

## Imports

In [1]:
# std lib
from getpass import getpass

# standard
import numpy as np
import pylab as plt
import matplotlib
%matplotlib inline
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import pandas as pd
from numpy.polynomial import polynomial as P

# Data Lab
from dl import authClient as ac, queryClient as qc, storeClient as sc

# others
from dl.helpers.utils import convert
import os
import glob
import gc

In [2]:
# Login

token = ac.login(input("Enter user name: (+ENTER) "),getpass("Enter password: (+ENTER) "))

Enter user name: (+ENTER)  malinadesai
Enter password: (+ENTER)  ········


## Get the DES Data

In [3]:
# get the des tile names
# you may need to change the file name to the file path pointing to UCD-Search/data/destilenames.csv

destilenames = pd.read_csv('destilenames.csv') 

In [4]:
# functions

def save_to_csv(frame, filename):
    ''' 
    Takes a dataframe [frame] and saves it as a csv with name [filename]
    frame     -   pd.dataframe object
    filename  -   string
    Adds the index to the dataframe
    '''
    frame['index'] = frame.index
    frame.to_csv(filename, index=False)
    
def save_to_csv_noindex(frame, filename):
    ''' 
    Takes a dataframe [frame] and saves it as a csv with name [filename]
    frame     -   pd.dataframe object
    filename  -   string
    Does not add the index to the dataframe
    '''
    frame.to_csv(filename, index=False) 
    
def get_tile_dbtable_vhs(table_num):
    '''
    table_num - integer representing tile number associated with desired table
    '''
    query = 'SELECT * from mydb://desvhs_tile{}'.format(str(table_num))
    df = convert(qc.query(sql=query))
    
    # Process the dataframe
    df = df.dropna(subset=['japermag3', 'japermag3err', 'ksapermag3', 'ksapermag3err'])
    df = df.reindex(df['dist_arcsec'].sort_values().index)
    duplicate = df[df.duplicated(subset=['sourceid'])]
    df = df.drop_duplicates(subset = ['sourceid'])
    df = df.reset_index(drop = True)
    
    return df, duplicate

    
def get_tile_dbtable_cat(table_num):
    '''
    table_num - integer representing tile number associated with desired table
    '''
    query = 'SELECT * from mydb://descat_tile{}'.format(str(table_num))
    df = convert(qc.query(sql=query))
    
    # Process the dataframe
    df = df.dropna(subset=['w1mpro', 'w1sigmpro', 'w2mpro', 'w2sigmpro'])
    df = df.reindex(df['dist_arcsec'].sort_values().index)
    duplicate = df[df.duplicated(subset=['source_id'])]
    df = df.drop_duplicates(subset = ['source_id'])
    df = df.reset_index(drop = True)
    
    return df, duplicate


In [ ]:
# splitting DES into large tiles based on tilename

output = pd.DataFrame()               # initialize an empty dataframe
current_row = 0                       # initialize row counter
total_rows = len(destilenames)        # total number of tilenames           
remaining_rows = total_rows           # keeping track of how many rows left
tile_number = 1                       # used for numbering outputs as large tiles

while remaining_rows > 0:             # while there are rows left
    tick_counter = 0                  # sets counts for every 50 rows
    while (tick_counter < 50 and tick_counter >= 0 and remaining_rows > 0):  # doing 50 rows at a time
        # querying DES:
        query = """
        SELECT ra, dec, wavg_mag_psf_r, wavg_mag_psf_i, wavg_mag_psf_z, wavg_mag_psf_y, 
        wavg_magerr_psf_r, wavg_magerr_psf_i, wavg_magerr_psf_z, wavg_magerr_psf_y, tilename
        FROM des_dr2.main as dr2
        WHERE tilename = '{}'
            AND dr2.EXTENDED_CLASS_COADD <= 1
            AND dr2.IMAFLAGS_ISO_I = 0
            AND dr2.IMAFLAGS_ISO_Z = 0
            AND dr2.FLAGS_I < 4 
            AND dr2.FLAGS_Z < 4
        """.format(destilenames.iloc[current_row, 0])
        # formatting response as a csv and turning it into a dataframe
        response = qc.query(sql=query,format='csv')
        sources = convert(response,'pandas')
        # removing -99's from required DES z and DES y band columns
        sources = sources[sources.wavg_mag_psf_z != -99.0]
        sources = sources[sources.wavg_mag_psf_y != -99.0] 
        # renaming columns 
        sources = sources.rename(columns = {'ra' : 'ra_des', 'dec' : 'dec_des'})
        # resetting index
        sources = sources.reset_index(drop = True)
        # concatenating all 50 results
        output = pd.concat([sources, output], ignore_index = True)    
        # changing relevant counters
        current_row += 1
        remaining_rows -=1
        tick_counter += 1
    # saving output as a csv file and re-initializing the output dataframe
    save_to_csv(output, 'des_tile%d.csv' % tile_number)
    tile_number += 1
    output = pd.DataFrame()


In [ ]:
# opening and processing des-vhs matches

desvhs_list = []

for i in range(1, 205):
    df, duplicate = get_tile_dbtable_vhs(i)
    desvhs_list.append([df, duplicate])
    
print(len(desvhs_list))


In [ ]:
# opening and processing des-cat matches

descat_list = []

for i in range(1, 205):
    df, duplicate = get_tile_dbtable_cat(i)
    descat_list.append([df, duplicate])
    
print(len(descat_list))

In [ ]:
# merge the des-vhs matches to the des-cat matches

for i in range(0, 204):
    
    if i == 29:
        pass
    if i != 29: 
        # perform the merge and save the result as a csv file
        df = pd.merge(desvhs_list[i][0], descat_list[i][0], on = 't1_index')
        save_to_csv_noindex(df, 'tile%d.csv' % i)

        # clearing out unnecessary variables and freeing memory
        vhs = desvhs_list[i][0]
        cat = descat_list[i][0]

        desvhs_list[i][0] = None
        descat_list[i][0] = None
    
        del(vhs)
        del(cat)
        gc.collect()